# Ollama Web Rag
## Config Web Loader

In [28]:
#https://python.langchain.com/docs/integrations/document_transformers/beautiful_soup/

from bs4 import BeautifulSoup
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION="python"


import os
from os.path import join, dirname
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import PGVector
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

OPENAI_API_KEY= os.environ.get('OPENAI_API_KEY')
PGVECTOR_CONNECTION_STRING = os.environ.get('PGVECTOR_CONNECTION_STRING')
USER_AGENT = os.environ.get('USER_AGENT')

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, Markdown



## Load Webpages

In [29]:
## Load Webpages
hn_loader =AsyncChromiumLoader(["https://news.ycombinator.com/"])
html = hn_loader.load()


RuntimeError: asyncio.run() cannot be called from a running event loop

In [15]:
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
    html, tags_to_extract=[".title .titleline a"]
)
print(docs_transformed)




Hacker News

Hacker News
new | past | comments | ask | show | jobs | submit 
login




1. Square Roots and Maxima (leancrew.com)
48 points by surprisetalk 3 hours ago  | hide | 11 comments 



2. Interview of Robert Shingledecker, Tiny Core Linux and DSL Developer (2009) (distrowatch.com)
39 points by transpute 4 hours ago  | hide | 7 comments 



3. Make the Most Of Your Burl (cindydrozda.com)
65 points by michael_forrest 6 hours ago  | hide | 20 comments 



4. Geometric line-art of Wacław Szpakowski (2017) (theparisreview.org)
302 points by bookofjoe 14 hours ago  | hide | 36 comments 



5. Rust-Query (lucasholten.com)
106 points by lukastyrychtr 10 hours ago  | hide | 44 comments 



6. Engineering Sleep (minjunes.ai)
296 points by amin 14 hours ago  | hide | 122 comments 



7. Breaking the 4Chan CAPTCHA (nullpt.rs)
445 points by hazebooth 23 hours ago  | hide | 259 comments 



8. What does this button do? – My new car has a mysterious and undocumented switch (koenvh.nl)
537 

In [4]:


# List of URLs to load documents from
urls = [
    "https://www.kodamakoifarm.com/koi-vs-carp/",
    "https://www.kodamakoifarm.com/japanese-temples-shrines-koi-pond-gardens/",
    "https://www.kodamakoifarm.com/goldfish/",
    "https://www.kodamakoifarm.com/koi-rehoming-koi-rescue-organizations/",
    "https://www.kodamakoifarm.com/taro-kodama-journey-into-world-of-koi-mastery/",
    "https://www.kodamakoifarm.com/koi-herpes-virus-khv-disease-guide/",
    "https://www.kodamakoifarm.com/sanitizing-letters/",
    "https://www.kodamakoifarm.com/request-koi-donation/",
    "https://www.kodamakoifarm.com/maintaining-and-fixing-pond-issues/",
    "https://www.lawnstarter.com/blog/landscaping/koi-pond-maintenance/",
    "https://www.thesprucepets.com/koi-pond-guide-5115233",
    "https://www.kodamakoifarm.com/new-look-same-great-koi/",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]


## Split text into chunks

In [5]:
#todo findout what tiktoken is text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
chunks = text_splitter.split_documents(docs_list)
print(f"Text split into {len(chunks)} chunks")

Text split into 432 chunks


## Vector Database

In [6]:
collection_name = "local-rag"

# chroma_db_openai = Chroma.from_documents(
#     documents=chunks,
#     embedding=OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY")),
#     collection_name=f"{collection_name}-openai"
# )
# 
# chroma_db_nomic = Chroma.from_documents(
#     documents=chunks,
#     embedding=OllamaEmbeddings(model="nomic-embed-text"),
#     collection_name= f"{collection_name}-nomic"
# )

pgvector_db_openai = PGVector.from_documents(
    collection_name=f"{collection_name}-openai",
    documents=chunks,
    embedding=OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY")),
    use_jsonb=True
)
pgvector_db_nomic = PGVector.from_documents(
    collection_name=f"{collection_name}-nomic",
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    use_jsonb=True
)

print(f"Vector database created successfully")

Vector database created successfully


## Create Chain

In [7]:
local_model = "llama3.2:latest"
#local_model = "granite3-dense:8b"
llm = ChatOllama(model=local_model)

query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant.  Your task is to generate 2 
    different versions of the give user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on user question, your
    goal is to help users overcome some of the limitations of distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}"""
)
# chroma_retriever_openai = MultiQueryRetriever.from_llm(
#     chroma_db_openai.as_retriever(),
#     llm,
#     prompt=query_prompt
# )
pg_retriever_openai = MultiQueryRetriever.from_llm(
    pgvector_db_openai.as_retriever(),
    llm,
    prompt=query_prompt
)
# chroma_retriever_nomic= MultiQueryRetriever.from_llm(
#     chroma_db_nomic.as_retriever(),
#     llm,
#     prompt=query_prompt
# )
pg_retriever_nomic = MultiQueryRetriever.from_llm(
    pgvector_db_nomic.as_retriever(),
    llm,
    prompt=query_prompt
)

In [9]:
template = """Answer the question on on the following context: 
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
# chroma_chain_openai = (
#     {"context": chroma_retriever_openai, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

pg_chain_openai = (
    {"context": pg_retriever_openai, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# chroma_chain_nomic = (
#     {"context": chroma_retriever_nomic, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

pg_chain_nomic = (
    {"context": pg_retriever_nomic, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# def chroma_chat_with_web_openai(question): 
#     """
#     Chat with the PDF using our chain
#     """
#     return display(Markdown(chroma_chain_openai.invoke(question)))
def pg_chat_with_web_openai(question): 
    """
    Chat with the PDF using our chain
    """
    return display(Markdown(pg_chain_openai.invoke(question)))
# def chroma_chat_with_web_nomic(question): 
#     """
#     Chat with the PDF using our chain
#     """
#     return display(Markdown(chroma_chain_nomic.invoke(question)))
def pg_chat_with_web_nomic(question): 
    """
    Chat with the PDF using our chain
    """
    return display(Markdown(pg_chain_nomic.invoke(question)))


In [115]:
chroma_chat_with_web_openai("What is prompt engineering?")

According to the provided documents, prompt engineering, also known as In-Context Prompting, refers to methods for communicating with LLMs (Large Language Models) to steer their behavior for desired outcomes without updating the model weights. It is an empirical science that requires heavy experimentation and heuristics, as the effect of prompt engineering methods can vary a lot among models.

In [116]:
chroma_chat_with_web_nomic("What is prompt engineering?")

According to the provided context, Prompt Engineering (also known as In-Context Prompting) refers to methods for communicating with LLMs to steer their behavior towards desired outcomes without updating the model weights. It is an empirical science that requires heavy experimentation and heuristics, and its effect can vary greatly among models.

In [14]:
pg_chat_with_web_openai("What virus's can a Koi catch?")

Based on the provided context, it appears that a Koi can catch Koi Herpes Virus (KHV). This is mentioned multiple times throughout the documents, specifically in relation to its symptoms, transmission patterns, and prevention measures.

In [16]:
pg_chat_with_web_nomic("What are steps for preventing herpes or treating KHV?")

According to the provided context, the article outlines several steps for preventing and managing Koi Herpes Virus (KHV):

Prevention:

1. Regular testing: Test your koi regularly for signs of KHV infection.
2. Biosecurity measures: Implement biosecurity measures to prevent the spread of KHV, such as maintaining optimal pond conditions.
3. Contact local vet or koi dealer first: If you suspect KHV infection, contact your local vet or koi dealer for guidance and testing.

Treatment:

1. Quarantine: Isolate infected koi to prevent further transmission.
2. Consult veterinarians: Seek professional advice from veterinarians experienced in treating KHV infections.

It is essential to note that the article emphasizes the importance of not jumping to conclusions about KHV infection without proper testing, as this can lead to unnecessary panic and overreaction.